In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Fashion Dataset v2.csv to Fashion Dataset v2.csv


In [ ]:
df = pd.read_csv("Fashion Dataset v2.csv")
df.head()

,p_id,name,products,price,colour,brand,img,ratingCount,avg_rating,description,p_attributes
0,17048614,Khushal K Women Black Ethnic Motifs Printed Ku...,"Kurta, Palazzos, Dupatta",5099.0,Black,Khushal K,http://assets.myntassets.com/assets/images/170...,4522.0,4.418399,Black printed Kurta with Palazzos with dupatta...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
1,16524740,InWeave Women Orange Solid Kurta with Palazzos...,"Kurta, Palazzos, Floral Print Dupatta",5899.0,Orange,InWeave,http://assets.myntassets.com/assets/images/165...,1081.0,4.119334,Orange solid Kurta with Palazzos with dupatta<...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
2,16331376,Anubhutee Women Navy Blue Ethnic Motifs Embroi...,"Kurta, Trousers, Dupatta",4899.0,Navy Blue,Anubhutee,http://assets.myntassets.com/assets/images/163...,1752.0,4.161530,Navy blue embroidered Kurta with Trousers with...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
3,14709966,Nayo Women Red Floral Printed Kurta With Trous...,"Kurta, Trouser, Dupatta",3699.0,Red,Nayo,http://assets.myntassets.com/assets/images/147...,4113.0,4.088986,Red printed kurta with trouser and dupatta<br>...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
4,11056154,AHIKA Women Black & Green Printed Straight Kurta,Kurta,1350.0,Black,AHIKA,http://assets.myntassets.com/assets/images/110...,21274.0,3.978377,"Black and green printed straight kurta, has a ...","{'Body Shape ID': '424', 'Body or Garment Size..."


In [ ]:
print(df.columns)

Index(['p_id', 'name', 'products', 'price', 'colour', 'brand', 'img',
       'ratingCount', 'avg_rating', 'description', 'p_attributes'],
      dtype='object')


In [ ]:
df['text'] = (
    df['products'].fillna('') + ' ' +
    df['brand'].fillna('') + ' ' +
    df['description'].fillna('')

)

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(df['text'].tolist(), show_progress_bar=True, normalize_embeddings=True)
np.save('fashion_embeddings.npy', embeddings)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/445 [00:00<?, ?it/s]

In [14]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 21.0 MB/s eta 0:00:00


In [15]:
import faiss

dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # cosine similarity
index.add(embeddings)

In [31]:
def search_products(query, top_k=5):
    query_vec = model.encode([query], normalize_embeddings=True)
    distances, indices = index.search(query_vec, top_k)
    results = df.iloc[indices[0]].copy()
    results['similarity'] = distances[0]
    return results[['products', 'brand', 'name', 'price', 'description', 'similarity']]


In [17]:
from transformers import pipeline

generator = pipeline("text-generation", model="tiiuae/falcon-7b-instruct", trust_remote_code=True)


config.json: 0.00B [00:00, ?B/s]

configuration_falcon.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



modeling_falcon.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

Device set to use cpu


In [28]:
def generate_recommendation(query, results):
    summary_text = f"User is looking for: {query}\nTop results:\n"
    for _, row in results.iterrows():
        summary_text += f"- {row['name']} by {row['brand']}, products: {row['products']}, price: ₹{row['price']}\n"

    prompt = f"Based on these options, suggest the best recommendations in a natural way:\n{summary_text}"
    response = generator(prompt, max_new_tokens=120, temperature=0.7)
    return response[0]['generated_text']



In [36]:
query = "comfortable cotton kurta for office wear"

results = search_products(query, top_k=5)

print(f"🛍️ Top Matches for: '{query}'\n")
display(results[['products', 'brand', 'name', 'price', 'description']])
recommendation_text = generate_recommendation(query, results)
print(recommendation_text)



🛍️ Top Matches for: 'comfortable cotton kurta for office wear'



,products,brand,name,price,description
525,Kurta,Libas,Libas Women Sea Green Ethnic Motifs Print V-Ne...,1399.0,Our sea-green straight-fit floral print cotton...
759,Kurta,Libas,Libas Women Peach Pink Solid Mandarin Collar C...,1299.0,Our peach solid coloured cotton straight-fit k...
531,"Kurta, Trousers",Anouk,Anouk Women Grey Printed Pure Cotton Kurta wit...,1699.0,Grey printed Kurta with Trousers <br> <br> <b>...
404,Kurta,Cot'N Soft,Cot'N Soft Women Black & White Striped Dobby K...,2499.0,<ul> <li> Colour: black and white </li> <li> S...
642,"Kurta, Trousers",Anouk,Anouk Women Purple Solid Kurta with Trousers,3399.0,Purple solid Kurta with Trousers <br> <br> <b>...
